In [ ]:
from cobra.io import read_sbml_model
%%capture
!carve --refseq GCF_000146045.2 --output Saccharomyces.xml --gapfill M9 --init M9

In [ ]:
model = read_sbml_model('Saccharomyces.xml')
print("Number of reactions:", len(model.reactions))
print("Number of metabolites:", len(model.metabolites))
print("Number of genes included in the model:", len(model.genes))
print("% of genes in model compared to the number of coding genes in Vibrio natrigens(4451 genes):",(len(model.genes)/4451)*100)

In [ ]:
!memote report snapshot --skip test_stoichiometric_consistency \
    --skip test_find_metabolites_not_produced_with_open_bounds \
    --skip test_find_metabolites_not_consumed_with_open_bounds Saccharomyces.xml --filename Saccharomyces.html

In [ ]:
!memote report snapshot Saccharomyces.xml --filename Saccharomyces.html

In [ ]:
!carve --refseq GCF_001456255.1 --output Test_Vnatri.xml --gapfill M9 --init M9

In [ ]:
!memote report snapshot Test_Vnatri.xml --filename Test_Vnatri.html